In [ ]:
!pip install mpi4py

In [ ]:
%%writefile primes.py
from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank ( )
size = comm.Get_size ( )
N = 108


def sives_primes(N, rank , size):
  sq_N =  int(N**0.5)
  if rank == 0:
    primes = np.ones(sq_N+1,dtype=bool)
    primes[0:2] = False
    for i in range(2, int(sq_N**0.5)+1):
      if primes[i]:
        primes[i * i : sq_N + 1 : i] = False

    s_primes = np.where(primes)[0]
  else :
    s_primes = None

  s_primes = MPI.COMM_WORLD.bcast(s_primes, root=0)
  low = sq_N + 1 + rank*(N - sq_N)//size
  high = sq_N + 1 + (rank+1)*(N - sq_N)//size
  segment = np.ones(high - low , dtype=bool )

  for prime in s_primes :
    batdau = max(prime*prime,(low + prime - 1)//prime * prime)
    segment[batdau - low: high - low : prime] = False

  local_primes = np.where(segment)[0]
  local_primes += low
  return local_primes

comm.Barrier()
thoigian_thucthi = MPI.Wtime()


local_primes = sives_primes(N, rank, size)

all_primes = comn.gather(local_primes,root = 0)
comm.Barrier()
thoigian_ketthuc = MPI.Wtime()
if rank == 0:
  all_primes = np.concatenate(all_primes)



  print(len(all_primes))
  print("Thoi gian chay",thoigian_ketthuc - thoigian_thucthi,"giay")
  print(all_primes)






In [ ]:
! mpirun --allow-run-as-root --oversubscribe -np 2 python primes.py